# (Optional) Testing the Function Endpoint with your Own Audio Clips


Instead of using pre-recorded clips we show you in this notebook how to invoke the deployed Function 
with your **own** audio clips. 

In the cells below, we will use the [PyAudio library](https://pypi.org/project/PyAudio/) to record a short 1 second clip. we will then submit 
that short clip to the Function endpoint on Oracle Functions. **Make sure PyAudio is installed on your laptop** before running this notebook. 

The helper function defined below will record a 1-sec audio clip when executed. Speak into the microphone 
of your computer and say one of the words `cat`, `eight`, `right`. 

I'd recommend double-checking that you are not muted and that you are using the internal computer mic. No 
headset.

In [ ]:
# we will use pyaudio and wave in the 
# bottom half of this notebook. 
import pyaudio
import wave

In [ ]:
print(pyaudio.__version__) 

In [ ]:
def record_wave(duration=1.0, output_wave='./output.wav'): 
    """Using the pyaudio library, this function will record a video clip of a given duration. 
    
    Args: 
        - duration (float): duration of the recording in seconds 
        - output_wave (str) : filename of the wav file that contains your recording 
        
    Returns: 
        - frames : a list containing the recorded waveform
    """
    
    # number of frames per buffer
    frames_perbuff = 2048 
    # 16 bit int
    format = pyaudio.paInt16
    # mono sound
    channels = 1 
    # Sampling rate -- CD quality (44.1 kHz). Standard 
    # for most recording devices. 
    sampling_rate = 44100 
    # frames contain the waveform data: 
    frames = []
    # number of buffer chunks: 
    nchunks = int(duration * sampling_rate / frames_perbuff)

    p = pyaudio.PyAudio()

    stream = p.open(format=format,
                channels=channels,
                rate=sampling_rate,
                input=True,
                frames_per_buffer=frames_perbuff) 
    
    print("RECORDING STARTED ")
    for i in range(0, nchunks):
        data = stream.read(frames_perbuff)
        frames.append(data)
    print("RECORDING ENDED")
    
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    # Write the audio clip to disk as a .wav file: 
    wf = wave.open(output_wave, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(sampling_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

In [ ]:
# let's record your own, 1-sec clip
my_own_clip = "./my_clip.wav"
frames = record_wave(output_wave=my_own_clip)

# Playback 
ipd.Audio("./my_clip.wav")

Looks good? Now let's try to send that clip to our model API endpoint. We will repeat the same process we adopted when we submitted pre-recorded clips.

In [2]:
# oci: 
import oci 
from oci.config import from_file
from oci import pagination
import oci.functions as functions
from oci.functions import FunctionsManagementClient, FunctionsInvokeClient

In [3]:
# Lets specify the location of our OCI configuration file: 
oci_config = from_file("/home/datascience/block_storage/.oci/config")

# Lets specify the compartment OCID, and the application + function names: 
compartment_id = 'ocid1.compartment.oc1..aaaaaaaafl3avkal72rrwuy4m5rumpwh7r4axejjwq5hvwjy4h4uoyi7kzyq' 
app_name = 'machine-learning-models'
fn_name = 'speech-commands'

In [5]:
fn_management_client = FunctionsManagementClient(oci_config)

app_result = pagination.list_call_get_all_results(
        fn_management_client.list_applications,
        compartment_id,
        display_name=app_name
    )

fn_result = pagination.list_call_get_all_results(
        fn_management_client.list_functions,
        app_result.data[0].id,
        display_name=fn_name
    )

invoke_client = FunctionsInvokeClient(oci_config, service_endpoint=fn_result.data[0].invoke_endpoint)

In [ ]:
# here we need to be careful. `my_own_clip` was recorded at a 44.1 kHz sampling rate. 
# Yet the training sample has data at a 16 kHz rate. To ensure that we feed data of the same 
# size, we will downsample the data to a 16 kHz rate (sr=16000)
waveform, _ = librosa.load(my_own_clip, mono=True, sr=16000)

Below we call the deployed Function. Note that the first call could take 60 sec. or more. This is due to the cold start problem of Function. Subsequent calls are much faster. Typically < 1 sec. 

In [ ]:
%%time

resp = invoke_client.invoke_function(fn_result.data[0].id, 
                                     invoke_function_body=json.dumps({"input": waveform.tolist()}))
print(resp.data.text)